***Welcome to Medical image processing in Python***<br/>

Presented by: Reza Saadatyar (2024-2025) <br/>
E-mail: Reza.Saadatyar@outlook.com 

**Import the require library**

In [15]:
import os
import sys
import glob
import random
import numpy as np
from tqdm import tqdm
from PIL import Image
from Functions import Data_path
import matplotlib.pyplot as plt
from keras.preprocessing import image
from skimage.transform import resize
from skimage import io
# from skimage.viewer import ImageViewer
# from OOP import Pre_Processing_R2g

**1. Set Image Path & Load data**

In [8]:
folder_path = "D:/Medical-Image-Processing/Data/Inputs_Train"
path_files, files, folders = Data_path.data_path(folder_path, data_format="tif") 

Image-Width: n<br/>
Image-Height: m<br/>
hannels: c<br/>
Planes: p<br/>
Grayscale:(p, m, n)<br/>
RGB: (p, m, n, c)<br/>

In [19]:
IMG_HEIGHT = 768
IMG_WIDTH = 896
IMG_CHANNELS = 3
Inputs = np.zeros((58, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype = np.uint8)   # 58*768*896*3
Ground_Truth = np.zeros((58, IMG_HEIGHT, IMG_WIDTH, 2), dtype = bool) #58*768*896*2

In [ ]:
imread(IMAGE_MASK_PATH + 'ytma10_010704_benign1.TIF')
    

In [ ]:
for n, f in tqdm(enumerate(IMG_Dataset), total = len(IMG_Dataset)):
    Images = imread (IMAGE_PATH + f)[:,:,:IMG_CHANNELS]